<a href="https://colab.research.google.com/github/seque1204/EduceLab/blob/main/DataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def get_centered_boudaries(image_shape, subarea_size, subareas_per_image):
  """
    Generate a list of bounding boxes centered around the middle of an image.

    Args:
        image_shape (tuple): A tuple representing the dimensions of the image (height, width, channels).
        subarea_size (tuple): A tuple representing the height and width of each subarea.
        subareas_per_image (int): The number of subareas to generate within the image.

    Returns:
        list: A list of tuples, each representing the bounding coordinates of a subarea within the image.
              Each tuple has the format (top_y, bottom_y, left_x, right_x).

    Functionality:
        - Calculates the center of the image using its dimensions.
        - Defines a standard deviation to control the spread of subarea locations around the center.
        - For each subarea, generates top-left coordinates based on a normal distribution centered around
          the image's midpoint.
        - Clips the coordinates to ensure each subarea is within the image boundaries.
        - Returns the list of boundaries, with each subarea centered around the middle, following a normal distribution.

    Example:
        If image_shape = (200, 200, 3), subarea_size = (50, 50), and subareas_per_image = 5,
        the function will return 5 bounding boxes focused around the center of the image, with each box
        sized 50x50 pixels.
  """

  height, width, channel = image_shape

  # Define the mean and normal distribution (center of image)
  mean_x = width // 2
  mean_y = height // 2

  # Define the standard deviation (smaller values will focus more on center)
  std_x = width // 8
  std_y = height // 8

  boundaries_list = []

  for _ in range(subareas_per_image):
    # Generate random values from the normal distribution
    top_left_x = int(np.clip(np.random.normal(loc = mean_x, scale = std_x), 0, width - subarea_size[1]))
    top_left_y = int(np.clip(np.random.normal(loc = mean_y, scale = std_y), 0, height - subarea_size[0]))
    boundaries_list.append((top_left_y, top_left_y + subarea_size[0], top_left_x, top_left_x + subarea_size[1]))
  return boundaries_list

In [2]:
def extract_subarea(image, boundaries):
  """
    Extract a subarea from an image based on given boundary coordinates.

    Args:
        image (numpy array): The input image array from which a subarea will be extracted.
                             Expected shape is (height, width, channels).
        boundaries (tuple): A tuple representing the coordinates of the subarea to extract.
                            The format should be (top_left_y, bottom_right_y, top_left_x, bottom_right_x),
                            where:
                              - top_left_y: The top boundary row index.
                              - bottom_right_y: The bottom boundary row index.
                              - top_left_x: The left boundary column index.
                              - bottom_right_x: The right boundary column index.

    Returns:
        numpy array: The extracted subarea as a slice of the original image, retaining
                     the same number of channels.

    Functionality:
        - Slices the input image array according to the specified boundaries.
        - Returns only the region within the defined boundaries, with the same number
          of color channels as the input image.

    Example:
        If boundaries = (50, 100, 30, 80), the function will return the portion of the
        image from row 50 to 100 and column 30 to 80.
    """

  top_left_y, bottom_right_y, top_left_x, bottom_right_x = boundaries
  subarea = image[top_left_y:bottom_right_y, top_left_x:bottom_right_x, :]
  return subarea

In [3]:
def subarea_generator(data_list, subarea_size=(32,32),subareas_per_image = 100):
  for data_item in data_list:
    #Extract image and label
    multi_channel_image = data_item['image_data']
    image_class = data_item['class']

    #generate centered boundaries
    boundaries_list = get_centered_boudaries(multi_channel_image.shape, subarea_size, subareas_per_image)

    #Yield each subarea and class as tensors
    for boundaries in boundaries_list:
      subarea = extract_subarea(multi_channel_image, boundaries)
      yield subarea, image_class

In [4]:
def create_dataset(data_list, subarea_size=(32,32),subareas_per_image = 10):
  """
    Create a TensorFlow dataset of image subareas using a generator function.

    Args:
        data_list (list): A list containing the image data or file paths to images. Each entry corresponds to an image.
        subarea_size (tuple, optional): The desired dimensions of each subarea (height, width). Defaults to (32, 32).
        subareas_per_image (int, optional): The number of subareas to generate per image. Defaults to 10.

    Returns:
        tf.data.Dataset: A TensorFlow dataset where each element is a tuple consisting of:
                         - A tensor of shape (subarea_height, subarea_width, 16) representing
                           a subarea of the original image.
                         - An integer label associated with the subarea, of type int32.
  """
  dataset = tf.data.Dataset.from_generator(
      lambda: subarea_generator(data_list, subarea_size, subareas_per_image),
      output_signature=(
          # The 16 here is the number of channels.
          tf.TensorSpec(shape=(subarea_size[0], subarea_size[1], 16), dtype=tf.float32),
          tf.TensorSpec(shape=(), dtype=tf.int32)
      )
  )

  return dataset


In [5]:
import numpy as np

def generate_data_list(num_images=10, image_shape=(64, 64, 16)):
    """
    Generate a list of images with specified dimensions, colors, and classes.

    Args:
        num_images (int): Total number of images to generate.
        image_shape (tuple): Dimensions of each image in (height, width, channels).

    Returns:
        list: A list of dictionaries, where each dictionary contains:
              - 'image_data': The image array filled with a specific color.
              - 'class': The class label based on color.
              - 'filename': A unique filename for each image.
    """
    # Define color vectors for each class
    colors = {
        1: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],  # Red
        2: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],  # Green
        3: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],  # Blue
        4: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   # Yellow
    }

    # Initialize data list
    data_list = []

    # Calculate the number of images per color
    images_per_color = num_images // len(colors)

    # Generate images
    for class_id, color_vector in colors.items():
        for i in range(images_per_color):
            image_data = np.tile(color_vector, (image_shape[0], image_shape[1], 1))  # Fill image with color
            filename = f"image_{class_id}_{i+1}.png"
            data_list.append({
                'image_data': image_data,
                'class': class_id - 1,
                'filename': filename
            })

    return data_list


In [10]:
import tensorflow as tf
data_list = generate_data_list()

batch_size = 32
subarea_size = (32,32)
channels = 16
train_dataset = create_dataset(data_list, subarea_size=subarea_size)
train_dataset = train_dataset.shuffle(buffer_size=100).batch(batch_size).prefetch(tf.data.AUTOTUNE)

#Define a simple CNN model for demo
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(subarea_size[0], subarea_size[1], channels)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  # 4 classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset, epochs=10)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.6109 - loss: 1.2394
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 0.6817
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss: 0.2561
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 0.0569
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 0.0058
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 4.9462e-04
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 5.7613e-05
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 1.5008e-05
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 5.4039e-06
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 2.3904e-06
